# Pipeline ETL - Sistema de Energia
## Extração, Transformação e Carga de Dados de Consumo

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import sqlite3
from src.utils import (
    gerar_dados_consumidores,
    gerar_dados_tarifas,
    gerar_dados_consumo,
    gerar_dados_picos
)

print(' Bibliotecas importadas com sucesso!')

 Bibliotecas importadas com sucesso!


## 1. EXTRAÇÃO (Extract)
### Gerando dados simulados

In [2]:
# Gerar dados de consumidores
df_consumidores = gerar_dados_consumidores(n=100)
print(f' Consumidores gerados: {len(df_consumidores)}')
df_consumidores.head()

 Consumidores gerados: 100


,id_consumidor,nome,tipo_consumidor,regiao,data_cadastro
0,1,Consumidor_1,Residencial,Norte,2023-01-01
1,2,Consumidor_2,Industrial,Oeste,2023-01-04
2,3,Consumidor_3,Comercial,Centro,2023-01-07
3,4,Consumidor_4,Residencial,Oeste,2023-01-10
4,5,Consumidor_5,Residencial,Centro,2023-01-13


In [3]:
# Gerar dados de tarifas
df_tarifas = gerar_dados_tarifas()
print(f' Tarifas geradas: {len(df_tarifas)}')
df_tarifas

 Tarifas geradas: 3


,id_tarifa,nome_tarifa,tipo_horario,valor_kwh,vigencia_inicio,vigencia_fim
0,1,Tarifa Pico,Pico,0.85,2024-01-01,2024-12-31
1,2,Tarifa Fora Pico,Fora_Pico,0.45,2024-01-01,2024-12-31
2,3,Tarifa Intermediária,Intermediario,0.65,2024-01-01,2024-12-31


In [11]:
# Gerar dados de consumo (30 dias)
df_consumo = gerar_dados_consumo(df_consumidores, dias=30)
print(f' Registros de consumo gerados: {len(df_consumo):,}')
df_consumo.head()

 Registros de consumo gerados: 72,000


,id_consumo,id_consumidor,data_hora,consumo_kwh,id_tarifa,custo_total
0,1,1,2024-10-01 00:00:00,5.12,2,2.30
1,2,1,2024-10-01 01:00:00,4.54,2,2.04
2,3,1,2024-10-01 02:00:00,4.19,2,1.89
3,4,1,2024-10-01 03:00:00,3.03,2,1.36
4,5,1,2024-10-01 04:00:00,3.03,2,1.36


In [12]:
# Gerar dados de picos
df_picos = gerar_dados_picos(dias=30)
print(f' Registros de picos gerados: {len(df_picos)}')
df_picos.head()

 Registros de picos gerados: 720


,id_pico,data_hora,demanda_total_mw,temperatura_celsius,dia_semana
0,1,2024-10-01 00:00:00,374.91,34.26,Tuesday
1,2,2024-10-01 01:00:00,446.40,28.98,Tuesday
2,3,2024-10-01 02:00:00,331.20,22.34,Tuesday
3,4,2024-10-01 03:00:00,311.62,32.99,Tuesday
4,5,2024-10-01 04:00:00,420.22,30.62,Tuesday


## 2. TRANSFORMAÇÃO (Transform)
### Limpeza e enriquecimento dos dados

In [6]:
# Converter data_hora para datetime
df_consumo['data_hora'] = pd.to_datetime(df_consumo['data_hora'])
df_picos['data_hora'] = pd.to_datetime(df_picos['data_hora'])

# Adicionar colunas derivadas
df_consumo['data'] = df_consumo['data_hora'].dt.date
df_consumo['hora'] = df_consumo['data_hora'].dt.hour
df_consumo['dia_semana'] = df_consumo['data_hora'].dt.day_name()
df_consumo['mes'] = df_consumo['data_hora'].dt.month

print(' Transformações aplicadas!')
df_consumo.head()

 Transformações aplicadas!


,id_consumo,id_consumidor,data_hora,consumo_kwh,id_tarifa,custo_total,data,hora,dia_semana,mes
0,1,1,2024-10-01 00:00:00,5.12,2,2.30,2024-10-01,0,Tuesday,10
1,2,1,2024-10-01 01:00:00,4.54,2,2.04,2024-10-01,1,Tuesday,10
2,3,1,2024-10-01 02:00:00,4.19,2,1.89,2024-10-01,2,Tuesday,10
3,4,1,2024-10-01 03:00:00,3.03,2,1.36,2024-10-01,3,Tuesday,10
4,5,1,2024-10-01 04:00:00,3.03,2,1.36,2024-10-01,4,Tuesday,10


In [7]:
# Verificar qualidade dos dados
print(' Verificação de Qualidade dos Dados\n')
print('Valores nulos:')
print(df_consumo.isnull().sum())
print('\nEstatísticas descritivas:')
df_consumo[['consumo_kwh', 'custo_total']].describe()

 Verificação de Qualidade dos Dados

Valores nulos:
id_consumo       0
id_consumidor    0
data_hora        0
consumo_kwh      0
id_tarifa        0
custo_total      0
data             0
hora             0
dia_semana       0
mes              0
dtype: int64

Estatísticas descritivas:


,consumo_kwh,custo_total
count,72000.000000,72000.000000
mean,34.386123,23.497055
std,65.899689,49.743674
min,1.510000,0.680000
25%,6.760000,4.020000
50%,12.600000,7.975000
75%,28.950000,20.332500
max,585.010000,497.260000


## 3. CARGA (Load)
### Salvando dados no banco SQLite

In [13]:
import os
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

# Conectar ao banco de dados
conn = sqlite3.connect('../data/energia.db')
print(' Conectado ao banco de dados')

# Carregar schema
with open('../sql/schema.sql', 'r') as f:
    schema = f.read()
    conn.executescript(schema)
    print(' Schema criado')

# Carregar dados nas tabelas
df_consumidores.to_sql('consumidores', conn, if_exists='replace', index=False)
print(f' {len(df_consumidores)} consumidores carregados')

df_tarifas.to_sql('tarifas', conn, if_exists='replace', index=False)
print(f' {len(df_tarifas)} tarifas carregadas')

df_consumo.to_sql('consumo', conn, if_exists='replace', index=False)
print(f' {len(df_consumo):,} registros de consumo carregados')

df_picos.to_sql('picos', conn, if_exists='replace', index=False)
print(f' {len(df_picos)} registros de picos carregados')

conn.close()
print('\n Pipeline ETL concluído com sucesso!')

 Conectado ao banco de dados
 Schema criado
 100 consumidores carregados
 3 tarifas carregadas
 72,000 registros de consumo carregados
 720 registros de picos carregados

 Pipeline ETL concluído com sucesso!


## 4. Salvar dados processados em CSV

In [14]:
# Salvar em CSV para análises futuras
df_consumo.to_csv('../data/processed/consumo_processado.csv', index=False)
df_consumidores.to_csv('../data/processed/consumidores.csv', index=False)
df_tarifas.to_csv('../data/processed/tarifas.csv', index=False)
df_picos.to_csv('../data/processed/picos.csv', index=False)

print(' Dados salvos em CSV!')

 Dados salvos em CSV!
